In [1]:
from sqlalchemy import Table, Column, Integer, String, MetaData, Date, Boolean, UniqueConstraint
from sqlalchemy.exc import IntegrityError
import sqlalchemy
import datetime 
from db.postgres.postgres_db import PostgresDB

In [2]:
# on a table I made in the 'testing' schema
engine = PostgresDB().engine
with PostgresDB().engine.connect() as engine:
    row = engine.execute('SELECT * FROM testing.test_table')
    print(row.keys())
    for r in row:
        print(r)

['id', 'name']
(1, 'Oranges')
(2, 'Bananas')
(3, 'Apples')
(4, 'Pears')


In [3]:
# creating a table
meta = MetaData(schema='testing')
table = Table(
    'generate_from_sqlalchemy', meta,
    Column('id', Integer, primary_key=True),
    Column('name', String, nullable=False),
    Column('created', Date, unique=True, nullable=False),
    Column('is_active', Boolean, nullable=False),
    UniqueConstraint('name', name='unameix_1')
    
    )
with PostgresDB().engine.connect() as engine:
    meta.create_all(PostgresDB().engine)

In [4]:
# throws a unique constraint exception
try:
    with PostgresDB().engine.connect() as engine:
        engine.execute(table.insert().values([
            {'name':'apples', 'created':datetime.datetime.now()-datetime.timedelta(5), 'is_active':True},
            {'name':'bananas', 'created':datetime.datetime.now()+datetime.timedelta(5), 'is_active':False},
            {'name':'pears', 'created':datetime.datetime.now()+datetime.timedelta(3), 'is_active':True}
            ]))
except IntegrityError as exc:
    print(exc)

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "unameix_1"
DETAIL:  Key (name)=(apples) already exists.

[SQL: INSERT INTO testing.generate_from_sqlalchemy (name, created, is_active) VALUES (%(name_m0)s, %(created_m0)s, %(is_active_m0)s), (%(name_m1)s, %(created_m1)s, %(is_active_m1)s), (%(name_m2)s, %(created_m2)s, %(is_active_m2)s)]
[parameters: {'name_m0': 'apples', 'created_m0': datetime.datetime(2020, 7, 8, 0, 55, 7, 156717), 'is_active_m0': True, 'name_m1': 'bananas', 'created_m1': datetime.datetime(2020, 7, 18, 0, 55, 7, 156726), 'is_active_m1': False, 'name_m2': 'pears', 'created_m2': datetime.datetime(2020, 7, 16, 0, 55, 7, 156727), 'is_active_m2': True}]
(Background on this error at: http://sqlalche.me/e/13/gkpj)


In [5]:
engine = PostgresDB().engine
with PostgresDB().engine.connect() as engine:
    row = engine.execute('SELECT * FROM testing.generate_from_sqlalchemy')
    print(row.keys())
    for r in row:
        print(r)

['id', 'name', 'created', 'is_active']
(1, 'oranges', datetime.date(2020, 7, 13), True)
(4, 'apples', datetime.date(2020, 7, 8), True)
(5, 'bananas', datetime.date(2020, 7, 18), False)
(6, 'pears', datetime.date(2020, 7, 16), True)
